# False Alarm Probability for CARMA(2,1) Process, INTEGRAL Data

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

import pickle
import numpy as np
import scipy.io

# Need both Jax and Tensorflow Probability 
import jax
# Important to enable 64-bit precision
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jax import random
import tensorflow_probability.substrates.jax as tfp

# TinyGP for Gaussian Processes, JaxNS for sampling
from tinygp import GaussianProcess, kernels
from tinygp.kernels.quasisep import CARMA

from jaxns.utils import resample

#stingray imports
import stingray
from stingray import Lightcurve, Powerspectrum
from stingray.modeling.gpmodeling import get_kernel, get_mean
from stingray.modeling.gpmodeling import get_prior, get_log_likelihood, get_gp_params

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# abbreviations for tensorflow distributions + bijectors
tfpd = tfp.distributions
tfpb = tfp.bijectors

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro.contrib.nested_sampling import NestedSampler

import arviz as az

INFO[2023-10-24 09:50:52,807]: Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
I0000 00:00:1698133852.807368       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
INFO[2023-10-24 09:50:52,808]: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO[2023-10-24 09:50:52,816]: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/opt/miniconda3/envs/grbsearch/bin/../lib/libtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file), '/usr/local/lib/libtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache)
/Users/daniela/work/repositories/stingraysoftware/stingray/stingray/utils.py:35: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn(

Let's load the real data:

In [2]:
def load_integral_data(fpath):
    """
    Load the Integral data from 
    the IDL .sav file. The file has two columns: 
    `barytime` and `counts`

    Parameters
    ----------
    fpath : str
        The path to the file with the data

    Returns
    -------
    lc : stingray.Lightcurve object
        The barycentred light curve
    """
    data = scipy.io.readsav(fpath)
    barytime = np.array(data["barytime"], dtype=float)
    counts = np.array(data["counts"], dtype=float)
    mean_bkg = np.mean(counts[-100:])

    lc = Lightcurve(barytime, counts-mean_bkg)
    minind = lc.time.searchsorted(0.0)
    maxind = lc.time.searchsorted(60.0)
    lc = lc.truncate(start=minind, stop=maxind, method="index")

    return lc


In [4]:
# load Integral data
datadir = "/Users/daniela/work/data/grb230307A/"
fpath = datadir + "acs_lc_bary.sav"
lc = load_integral_data(fpath)


INFO[2023-10-24 09:51:31,233]: Checking if light curve is well behaved. This can take time, so if you are sure it is already sorted, specify skip_checks=True at light curve creation.
INFO[2023-10-24 09:51:31,340]: Checking if light curve is sorted.
INFO[2023-10-24 09:51:31,622]: Computing the bin time ``dt``. This can take time. If you know the bin time, please specify it at light curve creation


Let's load the posterior:

In [25]:
def carma21_integral(t, y=None):

    mean_bkg = np.mean(y[-100:])
    # mean parameters
    log_amp = numpyro.sample('log_amp', dist.Uniform(9, 12.0))
    t0 = numpyro.sample("t0", dist.Uniform(0.0, 10))
    log_sig1 = numpyro.sample("log_sig1", dist.Uniform(-5, 1.5))
    log_sig2 = numpyro.sample("log_sig2", dist.Uniform(1, 4))
    log_const = numpyro.sample("log_const", dist.Normal(mean_bkg, mean_bkg/5))

    params = [log_amp, t0, log_sig1,log_sig2, log_const]
    
    mean = skewgaussian(t, params)
    
    # kernel parameters
    log_alpha1 = numpyro.sample("log_alpha1", dist.Uniform(-10, 20))
    log_alpha2 = numpyro.sample("log_alpha2", dist.Uniform(-10, 20))
    
    log_beta = numpyro.sample("log_beta", dist.Uniform(-10, 20))
        
    kernel = CARMA.init(alpha=[jnp.exp(log_alpha1), jnp.exp(log_alpha2)], 
                        beta=[jnp.exp(log_beta)])
    
    gp = GaussianProcess(kernel, t, diag=y, mean_value=mean)
        
    numpyro.sample("gp", gp.numpyro_dist(), obs=y)

    if y is not None:
        numpyro.deterministic("pred", gp.condition(y, t).gp.loc)

def sample_carma21(lc):
    """
    Set up a CARMA(2,1) + mean model and sample from 
    the posterior for a given Lightcurve.
    
    Parameters
    ----------
    lc : stingray.Lightcurve object
        The light curve to model with a DRW
        
    Returns
    -------
    gpresult_sim : stingray.modeling.GPResult object
        The object with the sampling results
    """
    
    ns = NestedSampler(carma21_integral)
    ns.run(random.PRNGKey(200), lc.time, y=lc.counts)   
    
    return ns

def simulate_grb(pars_log, time):
    """
    Simulate a GRB light curve given a set of parameters.
    
    Parameters
    ----------
    pars_log : dict
        A dictionary with the parameters for the 
        data generation process
        
    time : numpy.ndarray
        An array of time stamps
        
    Returns
    -------
    lcsim : stingray.Lightcurve
        A simulated light curve
    """
    dt = time[1]-time[0]
    
    p = 2
    q = 1
    mean_var_names = ["log_amp", "t0", "log_sig1",
                    'log_sig2', "log_const"]

    mean_params = [pars_log["log_amp"],
                   pars_log["t0"],
                   pars_log["log_sig1"],
                   pars_log["log_sig2"],
                   pars_log["log_const"]]

    alpha = [jnp.exp(pars_log["log_alpha1"]), jnp.exp(pars_log["log_alpha2"])]
    beta = [jnp.exp(pars_log["log_beta"])]

    mean_val = skewgaussian(time, mean_params)
    kernel = CARMA.init(alpha=alpha, beta=beta)

    gp = GaussianProcess(kernel, time, diag=0.01)

    key = random.PRNGKey(np.random.randint(0, 1000))
    sample = gp.sample(key) + mean_val
    
    sample = sample.at[sample <= 0].set(1e-10)
    sample = np.random.poisson(sample)
    lcsim = Lightcurve(time, sample, dt=dt, skip_checks=True)
    
    return lcsim

def skewgaussian(t, params):
    #parameter_names = ("logA", "t0", "logsig1", "logsig2", "logconst")
    logA = params[0]
    t0 = params[1]
    logsig1 = params[2]
    logsig2 = params[3]
    logconst = params[4]

    y = jnp.exp(logA) * jnp.where(
            t > t0,
            jnp.exp(-((t - t0) ** 2) / (2 * (jnp.exp(logsig2)**2))),
            jnp.exp(-((t - t0) ** 2) / (2 * (jnp.exp(logsig1)**2))),
        )
    return y + jnp.exp(logconst)


In [22]:
with open(datadir + "integral_carma21_ns.pkl", "rb") as f:
    ns = pickle.load(f)

samples, weights = ns.get_weighted_samples()

all_params = list(samples.keys())

all_params.remove("pred")

nsim = 100
nsamples = samples[list(samples.keys())[0]].shape[0]

idx_all = np.random.choice(np.arange(0, nsamples, 1, dtype=int), size=nsim, replace=False)

#lcsim_all = []

#carma21_logz_file = f"{datadir}integral_sim_carma21_logz.txt"
#qpo_logz_file = f"{datadir}integral_sim_carma21_qpo_logz.txt"

idx_all = [0]

In [23]:
all_params

['log_alpha1',
 'log_alpha2',
 'log_amp',
 'log_beta',
 'log_const',
 'log_sig1',
 'log_sig2',
 't0']

In [ ]:
for i, idx in enumerate(idx_all):
        print(f"I am on simulation {i}")
        pars_log = dict((k, samples[k][idx]) for k in all_params)

        #print(pars_log)
        lcsim = simulate_grb(pars_log, lc.time)
        
        #np.savetxt(f"{datadir}integral_lcsim{i}.dat", np.array([lcsim.time, lcsim.counts]).T)
        #lcsim_all.append(lcsim)
        
        print("Sampling the DRW ...")
        # sample the DRW
        ns_rn = sample_carma21(lcsim)
        
        #with open(f"{datadir}integral_carm21_res_sim{i}.pkl", "wb") as f:
        #    pickle.dump(ns_rn._results, f)
            
        #with open(carma21_logz_file, "a") as f:
        #    logz_drw = f"{ns_rn._results.log_Z_mean} \t {ns_rn._results.log_Z_uncert} \n"
        #    f.write(logz_drw)

I am on simulation 0
Sampling the DRW ...
